---
## Setup and Imports

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras

# Scikit-learn
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats

# Model interpretation
import shap

# Utilities
import joblib

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully!")

---
## Load Trained Models and Data

In [ ]:
# Define paths
models_dir = Path('models')
data_path = Path('project_data')
splits_path = data_path / 'train_test_split'

print("Loading trained models...")

# Load models
fnn_model = keras.models.load_model(models_dir / 'fnn_model.h5')
lstm_model = keras.models.load_model(models_dir / 'lstm_model.h5')
hybrid_model = keras.models.load_model(models_dir / 'hybrid_model.h5')

print("  ✓ Models loaded")

# Load scalers
fnn_scaler = joblib.load(models_dir / 'fnn_scaler.pkl')
lstm_scaler = joblib.load(models_dir / 'lstm_scaler.pkl')
hybrid_temp_scaler = joblib.load(models_dir / 'hybrid_temp_scaler.pkl')
hybrid_stat_scaler = joblib.load(models_dir / 'hybrid_stat_scaler.pkl')

print("  ✓ Scalers loaded")

# Load encoders
le_crop = joblib.load(models_dir / 'le_crop.pkl')
le_zone = joblib.load(models_dir / 'le_zone.pkl')
le_state = joblib.load(models_dir / 'le_state.pkl')

print("  ✓ Encoders loaded")

In [ ]:
# Load test data
print("\nLoading test datasets...")

fnn_test = pd.read_csv(splits_path / 'fnn' / 'test.csv')
lstm_test = pd.read_csv(splits_path / 'lstm' / 'test.csv')
hybrid_test = pd.read_csv(splits_path / 'hybrid' / 'test.csv')

# Remove missing yields
fnn_test = fnn_test.dropna(subset=['Yield_tonnes_per_ha'])
lstm_test = lstm_test.dropna(subset=['Yield_tonnes_per_ha'])
hybrid_test = hybrid_test.dropna(subset=['Yield_tonnes_per_ha'])

print(f"  FNN test: {fnn_test.shape}")
print(f"  LSTM test: {lstm_test.shape}")
print(f"  Hybrid test: {hybrid_test.shape}")

---
## 1. Regional Performance Analysis

### 1.1 Generate Predictions with Metadata

In [ ]:
# Prepare FNN predictions with metadata
print("Generating FNN predictions...")

# Define FNN features (same as Phase 3)
fnn_feature_cols = [
    'Avg_Temp_C', 'Min_Temp_C', 'Max_Temp_C', 'Temp_Range_C',
    'Rainfall_mm', 'Rainy_Days', 'Max_Daily_Rainfall_mm', 'Rainfall_Intensity',
    'Avg_Humidity_Percent', 'Min_Humidity_Percent', 'Max_Humidity_Percent',
    'CO2_ppm', 'CO2_Growth_Rate_ppm_per_year',
    'Heat_Stress_Days', 'Cold_Stress_Days', 'Drought_Index', 'Flood_Risk_Index',
    'Soil_pH', 'Organic_Matter_Percent', 'Nitrogen_ppm', 'Phosphorus_ppm', 
    'Potassium_ppm', 'Cation_Exchange_Capacity', 'Bulk_Density', 
    'Water_Holding_Capacity_Percent'
]

# Encode categoricals
fnn_test['Crop_encoded'] = le_crop.transform(fnn_test['Crop'])
fnn_test['Zone_encoded'] = le_zone.transform(fnn_test['Geopolitical_Zone'])
fnn_test['State_encoded'] = le_state.transform(fnn_test['State'])

fnn_feature_cols.extend(['Crop_encoded', 'Zone_encoded', 'State_encoded'])

# Prepare and predict
X_fnn_test = fnn_test[fnn_feature_cols].values
X_fnn_test_scaled = fnn_scaler.transform(X_fnn_test)
fnn_predictions = fnn_model.predict(X_fnn_test_scaled).flatten()

# Add predictions to dataframe
fnn_results = fnn_test[['Year', 'Geopolitical_Zone', 'State', 'Crop', 'Yield_tonnes_per_ha']].copy()
fnn_results['Predicted_Yield'] = fnn_predictions
fnn_results['Residual'] = fnn_results['Yield_tonnes_per_ha'] - fnn_results['Predicted_Yield']
fnn_results['Abs_Error'] = np.abs(fnn_results['Residual'])
fnn_results['Pct_Error'] = (fnn_results['Abs_Error'] / fnn_results['Yield_tonnes_per_ha']) * 100

print(f"  ✓ FNN predictions generated: {len(fnn_results)}")

### 1.2 Performance by Geopolitical Zone

In [ ]:
# Calculate metrics by zone
print("\n" + "="*80)
print("PERFORMANCE BY GEOPOLITICAL ZONE")
print("="*80)

zone_performance = fnn_results.groupby('Geopolitical_Zone').apply(
    lambda x: pd.Series({
        'N_samples': len(x),
        'RMSE': np.sqrt(mean_squared_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield'])),
        'MAE': mean_absolute_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'R2': r2_score(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'Mean_Actual': x['Yield_tonnes_per_ha'].mean(),
        'Mean_Predicted': x['Predicted_Yield'].mean(),
        'MAPE': x['Pct_Error'].mean()
    })
).round(4)

zone_performance = zone_performance.sort_values('R2', ascending=False)
print("\n", zone_performance.to_string())

In [ ]:
# Visualize zone performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('FNN Model Performance by Geopolitical Zone', fontsize=16, fontweight='bold')

zones = zone_performance.index

# R² by zone
axes[0,0].barh(zones, zone_performance['R2'], color='steelblue')
axes[0,0].set_xlabel('R² Score')
axes[0,0].set_title('R² Score by Zone')
axes[0,0].axvline(x=0.8, color='red', linestyle='--', alpha=0.5)
axes[0,0].grid(True, alpha=0.3, axis='x')

# RMSE by zone
axes[0,1].barh(zones, zone_performance['RMSE'], color='coral')
axes[0,1].set_xlabel('RMSE (tonnes/ha)')
axes[0,1].set_title('RMSE by Zone')
axes[0,1].grid(True, alpha=0.3, axis='x')

# MAE by zone
axes[1,0].barh(zones, zone_performance['MAE'], color='lightgreen')
axes[1,0].set_xlabel('MAE (tonnes/ha)')
axes[1,0].set_title('MAE by Zone')
axes[1,0].grid(True, alpha=0.3, axis='x')

# Sample size by zone
axes[1,1].barh(zones, zone_performance['N_samples'], color='gold')
axes[1,1].set_xlabel('Number of Samples')
axes[1,1].set_title('Test Samples by Zone')
axes[1,1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 1.3 Performance by State

In [ ]:
# Calculate metrics by state
state_performance = fnn_results.groupby('State').apply(
    lambda x: pd.Series({
        'Zone': x['Geopolitical_Zone'].iloc[0],
        'N_samples': len(x),
        'RMSE': np.sqrt(mean_squared_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield'])),
        'MAE': mean_absolute_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'R2': r2_score(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'Mean_Actual': x['Yield_tonnes_per_ha'].mean()
    })
).round(4)

state_performance = state_performance.sort_values('R2', ascending=False)

print("\n" + "="*80)
print("TOP 10 STATES BY MODEL PERFORMANCE")
print("="*80)
print(state_performance.head(10).to_string())

print("\n" + "="*80)
print("BOTTOM 10 STATES BY MODEL PERFORMANCE")
print("="*80)
print(state_performance.tail(10).to_string())

---
## 2. Crop-Specific Analysis

In [ ]:
# Performance by crop
print("\n" + "="*80)
print("PERFORMANCE BY CROP TYPE")
print("="*80)

crop_performance = fnn_results.groupby('Crop').apply(
    lambda x: pd.Series({
        'N_samples': len(x),
        'RMSE': np.sqrt(mean_squared_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield'])),
        'MAE': mean_absolute_error(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'R2': r2_score(x['Yield_tonnes_per_ha'], x['Predicted_Yield']),
        'Mean_Actual': x['Yield_tonnes_per_ha'].mean(),
        'Mean_Predicted': x['Predicted_Yield'].mean(),
        'MAPE': x['Pct_Error'].mean()
    })
).round(4)

crop_performance = crop_performance.sort_values('R2', ascending=False)
print("\n", crop_performance.to_string())

In [ ]:
# Visualize crop performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Performance by Crop Type', fontsize=16, fontweight='bold')

crops = crop_performance.index

# R² by crop
axes[0,0].barh(crops, crop_performance['R2'])
axes[0,0].set_xlabel('R² Score')
axes[0,0].set_title('R² Score by Crop')
axes[0,0].axvline(x=0.8, color='red', linestyle='--', alpha=0.5)
axes[0,0].grid(True, alpha=0.3, axis='x')

# RMSE by crop
axes[0,1].barh(crops, crop_performance['RMSE'], color='coral')
axes[0,1].set_xlabel('RMSE (tonnes/ha)')
axes[0,1].set_title('RMSE by Crop')
axes[0,1].grid(True, alpha=0.3, axis='x')

# Actual vs Predicted means
x = np.arange(len(crops))
width = 0.35
axes[1,0].barh(x - width/2, crop_performance['Mean_Actual'], width, label='Actual', alpha=0.8)
axes[1,0].barh(x + width/2, crop_performance['Mean_Predicted'], width, label='Predicted', alpha=0.8)
axes[1,0].set_yticks(x)
axes[1,0].set_yticklabels(crops)
axes[1,0].set_xlabel('Mean Yield (tonnes/ha)')
axes[1,0].set_title('Mean Actual vs Predicted Yield')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3, axis='x')

# MAPE by crop
axes[1,1].barh(crops, crop_performance['MAPE'], color='gold')
axes[1,1].set_xlabel('MAPE (%)')
axes[1,1].set_title('Mean Absolute Percentage Error')
axes[1,1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

---
## 3. Error Analysis

In [ ]:
# Residual analysis
residuals = fnn_results['Residual'].values

print("\n" + "="*80)
print("RESIDUAL ANALYSIS")
print("="*80)

print(f"\nResidual Statistics:")
print(f"  Mean: {residuals.mean():.4f}")
print(f"  Std Dev: {residuals.std():.4f}")
print(f"  Min: {residuals.min():.4f}")
print(f"  Max: {residuals.max():.4f}")
print(f"  Median: {np.median(residuals):.4f}")

# Test for normality
_, p_value = stats.normaltest(residuals)
print(f"\nNormality Test (p-value): {p_value:.4f}")
if p_value > 0.05:
    print("  → Residuals appear normally distributed")
else:
    print("  → Residuals deviate from normal distribution")

In [ ]:
# Residual diagnostic plots
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Residual Diagnostics', fontsize=16, fontweight='bold')

# Residuals vs Predicted
axes[0,0].scatter(fnn_results['Predicted_Yield'], residuals, alpha=0.5)
axes[0,0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,0].set_xlabel('Predicted Yield')
axes[0,0].set_ylabel('Residuals')
axes[0,0].set_title('Residuals vs Predicted')
axes[0,0].grid(True, alpha=0.3)

# Histogram of residuals
axes[0,1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[0,1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_xlabel('Residuals')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Distribution of Residuals')
axes[0,1].grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot')
axes[1,0].grid(True, alpha=0.3)

# Actual vs Predicted
axes[1,1].scatter(fnn_results['Yield_tonnes_per_ha'], fnn_results['Predicted_Yield'], alpha=0.5)
min_val = min(fnn_results['Yield_tonnes_per_ha'].min(), fnn_results['Predicted_Yield'].min())
max_val = max(fnn_results['Yield_tonnes_per_ha'].max(), fnn_results['Predicted_Yield'].max())
axes[1,1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
axes[1,1].set_xlabel('Actual Yield')
axes[1,1].set_ylabel('Predicted Yield')
axes[1,1].set_title('Actual vs Predicted')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Summary

In [ ]:
print("\n" + "="*80)
print("MODEL VALIDATION COMPLETE - SUMMARY")
print("="*80)

print("\n📊 OVERALL PERFORMANCE:")
print(f"  Test Period: 2020-2023")
print(f"  Samples: {len(fnn_results):,}")
overall_rmse = np.sqrt(mean_squared_error(fnn_results['Yield_tonnes_per_ha'], fnn_results['Predicted_Yield']))
overall_mae = mean_absolute_error(fnn_results['Yield_tonnes_per_ha'], fnn_results['Predicted_Yield'])
overall_r2 = r2_score(fnn_results['Yield_tonnes_per_ha'], fnn_results['Predicted_Yield'])
print(f"  RMSE: {overall_rmse:.4f} tonnes/ha")
print(f"  MAE: {overall_mae:.4f} tonnes/ha")
print(f"  R²: {overall_r2:.4f}")

print("\n🗺️ REGIONAL PERFORMANCE:")
best_zone = zone_performance.index[0]
worst_zone = zone_performance.index[-1]
print(f"  Best Zone: {best_zone}")
print(f"    R²: {zone_performance.loc[best_zone, 'R2']:.4f}")
print(f"  Worst Zone: {worst_zone}")
print(f"    R²: {zone_performance.loc[worst_zone, 'R2']:.4f}")

print("\n🌾 CROP-SPECIFIC PERFORMANCE:")
best_crop = crop_performance.index[0]
worst_crop = crop_performance.index[-1]
print(f"  Best Crop: {best_crop}")
print(f"    R²: {crop_performance.loc[best_crop, 'R2']:.4f}")
print(f"  Worst Crop: {worst_crop}")
print(f"    R²: {crop_performance.loc[worst_crop, 'R2']:.4f}")

print("\n✅ NEXT STEPS:")
print("  1. Review zone-specific and crop-specific performance")
print("  2. Investigate outliers and high-error predictions")
print("  3. Consider ensemble models for improved performance")
print("  4. Develop policy recommendations based on analysis")
print("  5. Create interactive dashboards for stakeholders")

print("\n" + "="*80)